## Model of Lupus Nephritis
#### see paper: [here](https://tbiomed.biomedcentral.com/articles/10.1186/1742-4682-7-14)

In [ ]:
import numpy as np

In [2]:
import numpy
# Define Variables and parameters
Lupusvars = list(var("I", "P", "D", "A"))

si = 0.002 #rate that immune complexes deposit in the kidneys
sid = 0.015 #immune response to accumulation of damaged cells
kid = 1
kip = 0.025 #rate of immune complex removal from system
kpi = 0.13 #rate of mediator activation and recruitment
kpp = 0.02 #immune response amplified by existing inflammatory response (kpi)
kpd = 0.001 #rate of activation for pro-inflammatory agents as a result of cytokine release or induced by damaged tissue
mup = 0.06 #decay of pro-inflammatory mediators
kdip = 0.025 #rate of phagocytosis of immune complexes by immune cells
kdp = 0.27 #rate at which collateral damage is produced by pro-inflammatory mediators 
mud = 0.04 #decay rate of damage
sa = 0.05 #addition of anti-inflammatory drugs
kap = 0.022 #intrarenal production of anti-inflammatory mediators
kad = 0.22 #intrarenal rate of tissue damage ??
mua = 2.2 #rate of anti-inflammatory agent degradation
Ainf = 0.45

(xmin, xmax, ymin, ymax) = (0, 1, 0, 4)

def f(x):
    return x/((1+A/Ainf)^2)

Idot = f(si) + f(sid)*(D^2/(kid^2 + D^2)) - kip*f(P)*I
Pdot = f(kpi*I + kpp*P) + f(kpd*D) - mup*P
Ddot = kdip*f(P)*I + kdp*f(P) - mud*D
Adot = sa + f(kap*P + kad*D) - mua*A

LupusSystem = (Idot, Pdot, Ddot, Adot)
init = (0.1, 0.4, 1.7, 0.1)

tmax=500
t_range = srange(0, tmax, 1)

LupusSim = desolve_odeint(LupusSystem, init, t_range, Lupusvars)
LupusSim = numpy.insert(LupusSim, 0, t_range, axis=1)
Its = list_plot(LupusSim[::,(0,1)], plotjoined=True, color="grey", legend_label="I")
Pts = list_plot(LupusSim[::,(0,2)], plotjoined=True, color="blue", legend_label="P")
Dts = list_plot(LupusSim[::,(0,3)], plotjoined=True, color="red", legend_label="D")
Ats = list_plot(LupusSim[::,(0,4)], plotjoined=True, color="black", legend_label="A")
show(Its+Pts+Dts+Ats, title = "Lupus", ymax = ymax)

1
2
3
4
5
